In [1]:
import pandas as pd
import time
from datetime import timedelta, datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Разделение пользователей по группам для рассылки

In [52]:
payments = pd.read_csv('users_courses__certificate_payments.csv')
payments.head()

,mipt_user_id,course_id,met_payment_condition,was_payment,was_finaid_grant,was_group_sponsored
0,74b19c8bcac1e1a2f6bbc57504e1fab9393d2e8f,RcnRZHHtEeWxvQr3acyajw,t,f,t,f
1,8a43b95a0e1e7995fbb7fc896d1ac0c4fbe5de30,RcnRZHHtEeWxvQr3acyajw,t,t,f,f
2,71fc78640755c1d77147d38caa7b2c7ad3095c72,RcnRZHHtEeWxvQr3acyajw,t,t,f,f
3,b9687946af32ca0e8f52a153f073e1b15af4c7d1,RcnRZHHtEeWxvQr3acyajw,t,f,f,t
4,b9d0f49f7bf359fca2189729149faa9386e54a81,RcnRZHHtEeWxvQr3acyajw,t,t,f,f


### Description from coursera's guide

The	*users_courses__certificate_payments*	table	provides	details	on	whether	an
enrollment	has	met	the	payment	condition	for	the	eligibility	of	earning	a	Course	Certificate. Below	are	the	three	paths	to	meet	the	payment	condition:

The	boolean	column,	 	*was_payment*	 ,	states	whether	the	learner	has	ever	paid	for	the
eligibility	of	a	course	certificate,	enrolled	in	the	course,	and	has	not	made	a	refund.	This
purchase	could	be	a	"single	payment"	for	the	course	or	a	"bulk	payment"	for	a
specialization	that	contains	the	course.	For	bulk	payments,	this	table	will	only	contain
rows	for	those	specalization's	courses	(or	capstone)	that	user	has	already	enrolled	in.
For	example,	a	learner	who	has	just	purchased	a	four-course	specialization	will	likely	be
enrolled	in	the	first	course,	which	adds	a	row	to	this	table.	Only	after	the	learner	enrolls
in	the	specialization's	second	course	will	a	second	row	appear.

The	boolean	column,	 	*was_finaid_grant*	 ,	states	whether	the	learner	has	ever	been
granted	financial	aid	for	the	course.	A	learner	can	only	apply	and	be	granted	financial
aid	to	courses	and	not	to	an	entire	specialization	at	once.	Of	note,	there	are	a	few	cases
where	the	learner	applied	and	was	granted	financial	aid	and	then	later	paid	for	the
course.

The	boolean	column,	 	*was_group_sponsored*	 ,	states	whether	the	learner	has	ever	been
sponsored	to	meet	the	payment	criteria	by	being	a	member	of	a	Coursera	group	which
has	the	premium	experience	enabled.

The	last	boolean	column,	 	*met_payment_condition*	 ,	states	whether	the	user	has	ever	met	the
payment	condition	to	be	eligible	for	the	course	certificate.	This	is	calculated	via	the	logical
'OR'	of	the	three	other	columns	above.

In [25]:
payments.shape

(3075, 6)

In [26]:
met = payments[payments['met_payment_condition'] == 't']
print met.shape # Получается,что все пользователи в этой таблице имеют доступ к курсу
met_payment_users = np.unique(met['mipt_user_id'].values)

(3075, 6)


In [27]:
was_payment = payments[payments['was_payment'] == 't']
was_payment_users = np.unique(was_payment['mipt_user_id'].values) #Юзеры, заплатившие за курс

In [29]:
prog = pd.read_csv('course_progress.csv')
prog.shape

(2364089, 5)

In [30]:
print len(np.unique(prog['mipt_user_id'].values)) #Количество всех уникальных юзеров,
                                                                    #проявивших активность на курсе

10690


In [31]:
prog_users = np.unique(prog['mipt_user_id'].values) #Юзеры, проявившие хоть какую-то активность на курсе

In [50]:
first_group_users = [] #Интересовались курсом, но не имеет к нему доступа
for user in prog_users:
    if not(user in met_payment_users):
        first_group_users.append(user)
print len(first_group_users)

7710


In [55]:
second_group_users = [] #Купили курс за деньги, но ни разу ничего на нём не делали
for user in was_payment_users:
    if not(user in prog_users):
        second_group_users.append(user)
print len(second_group_users)

25


In [56]:
course_branch_items = pd.read_csv('course_branch_items.csv') #Здесь содержится список заданий на этом курсе
course_item_types = pd.read_csv('course_item_types.csv') #Здесь можно посмотреть тип задания
course_item_types.head()

,course_item_type_id,course_item_type_desc,course_item_type_category,course_item_type_graded
0,1,lecture,lecture,f
1,7,others,NaN,NaN
2,10,graded programming,programming,t
3,8,graded peer,peer,t
4,9,closed peer,peer,t


In [57]:
course_branch_items.head()

,course_branch_id,course_item_id,course_lesson_id,course_branch_item_order,course_item_type_id,course_branch_item_name,course_branch_item_optional
0,RcnRZHHtEeWxvQr3acyajw,4JOsD,GjkOB,5,1,Направление наискорейшего роста,f
1,RcnRZHHtEeWxvQr3acyajw,TsbZO,AoSvR,11,3,Конспект,f
2,RcnRZHHtEeWxvQr3acyajw,MYHaa,GjkOB,0,1,Частные производные и градиент,f
3,RcnRZHHtEeWxvQr3acyajw,BoR1U,o6B9F,5,3,Конспект,f
4,RcnRZHHtEeWxvQr3acyajw,rcjAW,kEVhr,0,1,Pandas. Data Frame,f


In [58]:
#Создадим список заданий, которые будут являться индикаторами того,что человек выполняет оценочные задания
programming_types = course_branch_items[course_branch_items['course_item_type_id'] == 10]
graded_peer_types = course_branch_items[course_branch_items['course_item_type_id'] == 8]
peer_types = course_branch_items[course_branch_items['course_item_type_id'] == 4]
phased_peer = course_branch_items[course_branch_items['course_item_type_id'] == 12]
exam_quiz = course_branch_items[course_branch_items['course_item_type_id'] == 6]
practice_quiz = course_branch_items[course_branch_items['course_item_type_id'] == 5]
assessments = pd.concat([programming_types, graded_peer_types,phased_peer,peer_types,exam_quiz])
tasks = np.unique(assessments['course_item_id'].values)

In [59]:
indexes = [task in tasks for task in whole_tasks]

In [61]:
learner_prog = prog.iloc[indexes]
learners = np.unique(learner_prog['mipt_user_id'].values) #Юзеры,которые выполняли оценочные задания

In [54]:
lectures = course_branch_items[course_branch_items['course_item_type_id'] == 1]
lecture_items = np.unique(lectures['course_item_id'].values)
whole_tasks = list(prog['course_item_id'].values)
indexes = [task in lecture_items for task in whole_tasks]
lecture_prog = prog.iloc[indexes]
lecture_users = np.unique(lecture_prog['mipt_user_id'].values) #Юзеры,которые смотрели видео
len(lecture_users)

10414

In [62]:
third_group_users = [] #Купили курс,просматривали видео,но не приступали к оценочным заданиям
for user in was_payment_users:
    if not(user in learners) and (user in lecture_users):
        third_group_users.append(user)

In [63]:
print len(third_group_users), len(second_group_users)

126 25


In [65]:
course_grades = pd.read_csv('course_grades.csv')
course_grades.head()

,course_id,mipt_user_id,course_grade_ts,course_passing_state_id,course_grade_overall_passed_items,course_grade_overall,course_grade_verified_passed_items,course_grade_verified
0,RcnRZHHtEeWxvQr3acyajw,0073db5ea45438901c26a2ef08c2db9c8394c06f,2016-04-04 16:51:59.323,2,14,1.000000,14,1.000000
1,RcnRZHHtEeWxvQr3acyajw,947ddd151ab1aaa616ee74693f5d0bb9392d573b,2016-05-03 04:43:23.695,2,14,0.978444,14,0.978444
2,RcnRZHHtEeWxvQr3acyajw,1d0b3ef69e8687bd689bc9e1d76cd86e8648eafc,2016-03-07 01:10:07.052,2,14,0.975000,14,0.975000
3,RcnRZHHtEeWxvQr3acyajw,14cd3490328aef55a631676f59b65fb34be07148,2016-04-03 11:14:24.611,0,13,0.861052,13,0.861052
4,RcnRZHHtEeWxvQr3acyajw,e3cf7a3f2ac426421b76233c9d8c0a81c6afc0b8,2016-07-29 14:26:44.051,2,14,1.000000,14,1.000000


In [66]:
#Найдём юзеров,которые не смогли полностью пройти курс
not_passed_course_grades = course_grades[course_grades['course_passing_state_id'] == 0]
not_passed_course_users = np.unique(not_passed_course_grades['mipt_user_id'].values)

In [67]:
fourth_group_users = [] #Купили курс,начали выполнение заданий, но "отвалились"
for user in was_payment_users:
    if user in learners and user in not_passed_course_users:
        fourth_group_users.append(user)

In [68]:
len(fourth_group_users)

671

In [69]:
#Следующая задача состоит в том,чтобы найти пользователей которые 1) Купили 1 курс,но перестали покупать
                                                                # 2) Купили несколько,но перестали покупать
payments_1 = pd.read_csv('users_courses__certificate_payments(1).csv')
payments_2 = pd.read_csv('users_courses__certificate_payments(2).csv')
payments_3 = pd.read_csv('users_courses__certificate_payments(3).csv')
payments_4 = pd.read_csv('users_courses__certificate_payments(4).csv')

In [70]:
paid_1 = payments_1[payments_1['was_payment'] == 't']
paid_2 = payments_2[payments_2['was_payment'] == 't']
paid_3 = payments_3[payments_3['was_payment'] == 't']
paid_4 = payments_4[payments_4['was_payment'] == 't']
users_1 = np.unique(paid_1['mipt_user_id'].values)
users_2 = np.unique(paid_2['mipt_user_id'].values)
users_3 = np.unique(paid_3['mipt_user_id'].values)
users_4 = np.unique(paid_4['mipt_user_id'].values)

In [72]:
print len(users_1), len(users_2), len(users_3), len(users_4)
all_users = np.unique(np.concatenate([users_1,users_2,users_3,users_4]))

1492 630 320 222


In [75]:
#Этот кусок кода можно пропустить.Здесь лишь показано,что существуют юзеры,которые покупают курс не по порядку
###
try_users = np.concatenate([users_2,users_3,users_4])
groups = [users_1,users_2,users_3,users_4]
try_users = np.unique(try_users)
len(try_users)
k = 0
for user in try_users:
    if not user in users_1:
        k += 1
print k

30


In [76]:
pay_dict = {}
for user in all_users:
    pay_dict[user] = 0
for group in groups:
    for user in group:
        k = pay_dict[user]
        pay_dict[user] = k + 1

In [81]:
single = [] #Купили один курс,но перестали покупать
for user in pay_dict:
    if pay_dict[user] == 1:
        single.append(user)

In [82]:
multiple = [] #Купили несколько курсов,но перестали покупать
for user in pay_dict:
    if pay_dict[user] == 2 or pay_dict[user] == 3:
        multiple.append(user)

In [83]:
print len(single), len(multiple)

898 425


In [89]:
id_1_df = pd.DataFrame()
id_2_df = pd.DataFrame()
id_3_df = pd.DataFrame()
id_4_df = pd.DataFrame()
id_5_df = pd.DataFrame()
id_6_df = pd.DataFrame()

In [90]:
id_1_df['id_1'] = first_group_users
id_2_df['id_2'] = second_group_users
id_3_df['id_3'] = third_group_users
id_4_df['id_4'] = fourth_group_users
id_5_df['id_5'] = single
id_6_df['id_6'] = multiple

In [92]:
id_1_df.to_csv('id_1.csv')
id_2_df.to_csv('id_2.csv')
id_3_df.to_csv('id_3.csv')
id_4_df.to_csv('id_4.csv')
id_5_df.to_csv('id_5.csv')
id_6_df.to_csv('id_6.csv')